# Dekoratorji

V Pythonu lahko definiramo funkcijo, ki vrača funkcijo.

In [ ]:
import time
from functools import wraps

def stopaj(fun):
    @wraps(fun)
    def stopaj_fun(*largs, **kwargs):
        zacetek = time.time()
        rez = fun(*largs, **kwargs)
        konec = time.time()
        args = ', '.join((*map(str, largs), *(f'{k}={v}' for k, v in kwargs.items())))
        print(f"Klic funkcije {fun.__name__}({args}) je trajal {1000 * (konec - zacetek):.2f} ms.")
        return rez
    return stopaj_fun

In [ ]:
@stopaj
def vsota_pocasna(n):
    """
    Izračunaj 1 + 2 + ... + n s prištevanjem v zanki for.
    """
    vsota = 0
    for i in range(n+1):
        vsota += i
    return vsota

In [ ]:
@stopaj
def vsota_hitra(n):
    """
    Izračunaj 1 + 2 + ... + n s formulo n*(n+1)/2.
    """
    return (n * (n+1)) // 2

In [ ]:
@stopaj
def primerjaj(m, n):
    return vsota_hitra.__wrapped__(m) == vsota_pocasna.__wrapped__(n)

In [ ]:
vsota_pocasna(10**8)

In [ ]:
vsota_hitra(10**8)

In [ ]:
primerjaj(n=10**6, m=10**7)

In [ ]:
vsota_pocasna??

# Knjižnica `bottle`

Zapišimo vzorčni spletni vmesnik s knjižnico `bottle`.

In [2]:
import bottle
import os

SKRIVNOST = 'nekaj, kar bo zelo težko uganiti!!!! djnskfndkjfnsd'
BOTTLE_PORT = 8080
PREFIX = os.environ.get('JUPYTERHUB_SERVICE_PREFIX')
APP_PREFIX = f'{PREFIX}/proxy/{BOTTLE_PORT}' if PREFIX else ''

@bottle.get('/static/<datoteka:path>')
def static(datoteka):
    return bottle.static_file(datoteka, root='static')

@bottle.get('/')
def index():
    uporabnik = bottle.request.get_cookie('uporabnik', secret=SKRIVNOST)
    return bottle.template('index.html', uporabnik=uporabnik)

@bottle.get('/pozdravi/<ime>/')
@bottle.get('/pozdravi/<ime>/<priimek>/')
def pozdravi(ime, priimek=None):
    imepriimek = ime if priimek is None else f'{ime} {priimek}'
    return f'Živjo, <strong>{imepriimek}</strong>!'

@bottle.get('/kvadriraj/<n:float>')
@bottle.get('/kvadriraj/<n:int>')
def kvadriraj(n):
    return f'{n}^2 = {n**2}'

@bottle.get('/pozdravi_predloga/<ime>/')
@bottle.view('pozdravi.html')
def pozdravi_predloga(ime):
    return dict(ime=ime)

@bottle.get('/primer')
@bottle.get('/primer/')
def primer():
    return bottle.template('primer.html')

@bottle.get('/obrazec')
def obrazec():
    uporabnik = bottle.request.get_cookie('uporabnik', secret=SKRIVNOST)
    return bottle.template('obrazec.html', uporabnik=uporabnik)

@bottle.post('/obrazec/')
def obrazec_post():
    uporabnik = bottle.request.forms.uporabnik
    bottle.response.set_cookie('uporabnik', uporabnik, secret=SKRIVNOST, path='/')
    bottle.redirect(APP_PREFIX + '/obrazec')

@bottle.post('/odjava/')
def odjava():
    bottle.response.delete_cookie('uporabnik', path='/')
    bottle.redirect(APP_PREFIX + '/obrazec')

@bottle.get('/sestej/')
def sestej():
    try:
        a = int(bottle.request.query.a)
        b = int(bottle.request.query.b)
        return f'{a} + {b} = {a + b}'
        #bottle.redirect(APP_PREFIX + f'/kvadriraj/{a + b}')
    except ValueError as ex:
        bottle.abort(400, f"Vnesi dve celi števili! Napaka: {type(ex)}")

bottle.BaseTemplate.defaults['povecaj'] = lambda x: x+1

if PREFIX:
    from IPython.display import display, Markdown
    display(Markdown(f"[Klikni za zagon aplikacije]({APP_PREFIX}/)"))
bottle.run(debug=True, port=BOTTLE_PORT)

Bottle v0.14-dev server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [22/Dec/2025 10:42:41] "GET /sestej/?a=23&b=42 HTTP/1.1" 200 12
127.0.0.1 - - [22/Dec/2025 10:42:54] "GET /sestej/?a=23&b=42i HTTP/1.1" 400 797
127.0.0.1 - - [22/Dec/2025 10:45:13] "GET /sestej/?a=23%26&b=42%3D HTTP/1.1" 400 802
127.0.0.1 - - [22/Dec/2025 10:47:57] "POST /obrazec/ HTTP/1.1" 303 0
127.0.0.1 - - [22/Dec/2025 10:47:57] "GET /obrazec HTTP/1.1" 200 773
127.0.0.1 - - [22/Dec/2025 10:47:57] "GET /static/css/stil.css HTTP/1.1" 304 0
127.0.0.1 - - [22/Dec/2025 10:55:30] "POST /obrazec/ HTTP/1.1" 303 0
127.0.0.1 - - [22/Dec/2025 10:55:30] "GET /obrazec HTTP/1.1" 200 773
127.0.0.1 - - [22/Dec/2025 10:55:30] "GET /static/css/stil.css HTTP/1.1" 304 0
127.0.0.1 - - [22/Dec/2025 11:04:00] "POST /odjava/ HTTP/1.1" 303 0
127.0.0.1 - - [22/Dec/2025 11:04:00] "GET /obrazec HTTP/1.1" 200 753
127.0.0.1 - - [22/Dec/2025 11:04:00] "GET /static/css/stil.css HTT

In [ ]:
try:
    bottle.abort(400, "Vnesi dve celi števili!")
    print("konec")
except:
    print("zgodila se je izjema")